In [1]:
%pip install "scikit-learn<1.6"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 44.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


In [1]:
import pandas as pd
df = pd.read_csv('/content/sample_data/11_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [2]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.8 MB/s eta 0:00:00


In [3]:
%pip install lightbgm

ERROR: Could not find a version that satisfies the requirement lightbgm (from versions: none)
ERROR: No matching distribution found for lightbgm


LGBM

In [4]:
from lightgbm import LGBMRegressor
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost
from sklearn.metrics import make_scorer, mean_squared_error

candidate = LGBMRegressor(random_state=42)

param_distribs = {
    'n_estimators': randint(low=50, high=120),
    'max_depth': randint(low=6, high=12),
}

# Define the scoring function
scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

param_search = RandomizedSearchCV(
    candidate,
    param_distributions=param_distribs,
    n_iter=15,
    random_state=42,
    scoring=scorer,
    cv=3,
    n_jobs=-1
    )

param_search.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.168065 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1941
[LightGBM] [Info] Number of data points in the train set: 960000, number of used features: 54
[LightGBM] [Info] Start training from score 6.590488


RandomizedSearchCV(cv=3, estimator=LGBMRegressor(random_state=42), n_iter=15,
                   n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7acc3ab03670>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7acc54756800>},
                   random_state=42,
                   scoring=make_scorer(mean_squared_error, greater_is_better=False, response_method='predict', squared=False))

In [18]:
import pandas as pd
pd.set_option('display.max_colwidth', 10)
param_search.cv_results_ = pd.DataFrame(param_search.cv_results_)
param_search.cv_results_.sort_values(by=['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
13,19.209286,0.805701,3.071938,0.878799,11,100,{'max_...,-1.052544,-1.051552,-1.051711,-1.051936,0.000435,1
9,19.005620,0.718334,3.215643,0.767445,11,98,{'max_...,-1.052532,-1.051549,-1.051736,-1.051939,0.000426,2
12,19.868510,0.407030,3.249318,0.924736,11,111,{'max_...,-1.052587,-1.051561,-1.051697,-1.051948,0.000455,3
2,16.700860,1.750941,2.927910,0.493409,10,73,{'max_...,-1.052587,-1.051583,-1.051834,-1.052001,0.000427,4
14,18.165342,1.872704,2.455922,0.329076,9,104,{'max_...,-1.052635,-1.051671,-1.051751,-1.052019,0.000437,5
6,13.272809,1.459294,2.394657,0.602874,11,51,{'max_...,-1.052600,-1.051647,-1.051810,-1.052019,0.000416,6
4,13.040014,1.059648,2.597621,0.606116,10,51,{'max_...,-1.052692,-1.051629,-1.051831,-1.052051,0.000461,7
10,19.641028,0.292113,3.462677,1.070428,8,108,{'max_...,-1.052679,-1.051709,-1.051814,-1.052067,0.000434,8
5,16.664570,0.392143,2.798667,0.926621,9,79,{'max_...,-1.052637,-1.051747,-1.051819,-1.052068,0.000404,9
1,20.527165,0.912994,3.712829,0.481556,8,110,{'max_...,-1.052682,-1.051714,-1.051817,-1.052071,0.000434,10


In [6]:
param_search.best_params_

{'max_depth': 11, 'n_estimators': 100}

In [7]:
best_model = param_search.best_estimator_

In [8]:
from sklearn.metrics import root_mean_squared_error
y_pred = best_model.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0490192911005893

In [9]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [10]:
y_pred = best_model.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.0524820451518588

xgboost

In [19]:
candidate = xgb.XGBRegressor(random_state=42)

param_distribs = {
    'max_depth': randint(low=6, high=16),
    'n_estimators': randint(low=30, high=100),
}

# Define the scoring function
scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

param_search = RandomizedSearchCV(
    candidate,
    param_distributions=param_distribs,
    n_iter=15,
    random_state=42,
    scoring=scorer,
    cv=3,
    n_jobs=-1
    )

param_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          random_state=42, ...),
                   n_iter=15, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7acc54755f60>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7acc547555d0>},
                   random_state=42,
                   scoring=make_scorer(mean_squared_error, greater_is_better=False, response_method='predict', squared=False))

In [20]:
pd.set_option('display.max_colwidth', 10)
param_search.cv_results_ = pd.DataFrame(param_search.cv_results_)
param_search.cv_results_.sort_values(by=['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
3,20.741142,1.471700,1.560039,0.293214,8,51,{'max_...,-1.056748,-1.055844,-1.055634,-1.056075,0.000484,1
11,32.680973,7.212758,3.375670,1.785342,8,84,{'max_...,-1.059806,-1.058952,-1.058609,-1.059123,0.000503,2
4,18.912064,0.455738,1.636591,0.324744,10,31,{'max_...,-1.061771,-1.060763,-1.061374,-1.061302,0.000414,3
12,42.501164,3.876118,4.069118,1.557869,9,93,{'max_...,-1.066733,-1.065737,-1.066072,-1.066181,0.000414,4
6,21.805488,1.118987,2.219204,0.464603,11,31,{'max_...,-1.066830,-1.066045,-1.065974,-1.066283,0.000388,5
2,25.238193,1.584054,2.278582,0.522153,10,53,{'max_...,-1.066557,-1.066271,-1.066534,-1.066454,0.000130,6
7,28.194412,1.143099,3.899600,0.332362,10,62,{'max_...,-1.068544,-1.068613,-1.068633,-1.068596,0.000038,7
14,41.521517,1.057864,4.700868,0.847755,10,80,{'max_...,-1.072570,-1.073008,-1.072897,-1.072825,0.000186,8
13,46.242593,0.342417,4.301810,0.130083,14,32,{'max_...,-1.090781,-1.091169,-1.091741,-1.091230,0.000394,9
0,57.578357,3.550837,6.788255,1.136474,12,81,{'max_...,-1.095531,-1.094644,-1.094717,-1.094964,0.000402,10


In [21]:
param_search.best_params_

{'max_depth': 8, 'n_estimators': 51}

In [22]:
best_model = param_search.best_estimator_

In [23]:
from sklearn.metrics import root_mean_squared_error
y_pred = best_model.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0285523080024879

In [24]:
y_pred = best_model.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.0553019566908766

Catboost

In [25]:
candidate = Catboost(random_state=42)

param_distribs = {
    'iterations': randint(low=6, high=80),
    'depth': randint(low=5, high=25),
}

# Define the scoring function
scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

param_search = RandomizedSearchCV(
    candidate,
    param_distributions=param_distribs,
    n_iter=15,
    random_state=42,
    scoring=scorer,
    cv=3,
    n_jobs=-1
    )

param_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
15 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", l

Learning rate set to 0.5
0:	learn: 1.0727067	total: 278ms	remaining: 15.3s
1:	learn: 1.0636442	total: 483ms	remaining: 13.1s
2:	learn: 1.0590287	total: 681ms	remaining: 12s
3:	learn: 1.0571664	total: 860ms	remaining: 11.2s
4:	learn: 1.0559723	total: 1.05s	remaining: 10.8s
5:	learn: 1.0552606	total: 1.25s	remaining: 10.4s
6:	learn: 1.0549803	total: 1.47s	remaining: 10.3s
7:	learn: 1.0545039	total: 1.79s	remaining: 10.8s
8:	learn: 1.0542489	total: 2.14s	remaining: 11.2s
9:	learn: 1.0541359	total: 2.54s	remaining: 11.7s
10:	learn: 1.0539593	total: 2.92s	remaining: 11.9s
11:	learn: 1.0537823	total: 3.3s	remaining: 12.1s
12:	learn: 1.0535411	total: 3.62s	remaining: 12s
13:	learn: 1.0534816	total: 3.92s	remaining: 11.8s
14:	learn: 1.0533097	total: 4.28s	remaining: 11.7s
15:	learn: 1.0532210	total: 4.67s	remaining: 11.7s
16:	learn: 1.0531483	total: 5.06s	remaining: 11.6s
17:	learn: 1.0530586	total: 5.46s	remaining: 11.5s
18:	learn: 1.0529600	total: 5.82s	remaining: 11.3s
19:	learn: 1.0528911	

RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostRegressor object at 0x7acc3aab8310>,
                   n_iter=15, n_jobs=-1,
                   param_distributions={'depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7acc3aabb880>,
                                        'iterations': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7acc3aabb100>},
                   random_state=42,
                   scoring=make_scorer(mean_squared_error, greater_is_better=False, response_method='predict', squared=False))

In [26]:
pd.set_option('display.max_colwidth', 10)
param_search.cv_results_ = pd.DataFrame(param_search.cv_results_)
param_search.cv_results_.sort_values(by=['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_depth,param_iterations,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
13,22.957673,2.996995,0.376555,0.008650,7,56,{'dept...,-1.054362,-1.053469,-1.053520,-1.053783,0.000409,1
5,9.220088,1.566658,0.156918,0.011709,10,7,{'dept...,-1.054498,-1.053289,-1.053854,-1.053880,0.000494,2
4,11.368874,0.945147,0.258574,0.033186,6,35,{'dept...,-1.054280,-1.053628,-1.054131,-1.054013,0.000279,3
6,19.189337,0.688414,0.433065,0.116823,5,63,{'dept...,-1.054524,-1.053677,-1.054036,-1.054079,0.000347,4
3,10.821505,0.966519,0.381908,0.128542,7,27,{'dept...,-1.054915,-1.053928,-1.054392,-1.054411,0.000403,5
14,28.547104,7.597774,0.340934,0.107726,11,26,{'dept...,-1.057403,-1.057768,-1.057121,-1.057431,0.000265,6
2,35.824203,0.683457,0.434357,0.006081,11,29,{'dept...,-1.057839,-1.058128,-1.057455,-1.057807,0.000276,7
0,69.180093,2.777349,0.503652,0.042180,11,57,{'dept...,-1.062198,-1.062396,-1.061358,-1.061984,0.000450,8
8,209.53...,5.411693,0.768816,0.006324,14,65,{'dept...,-1.092189,-1.089693,-1.092342,-1.091408,0.001215,9
7,560.98...,2.302287,0.826224,0.071312,16,54,{'dept...,-1.114280,-1.115232,-1.114853,-1.114788,0.000391,10


In [27]:
param_search.best_params_

{'depth': 7, 'iterations': 56}

In [28]:
best_model = param_search.best_estimator_

In [29]:
y_pred = best_model.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0497097582909078

In [30]:
y_pred = best_model.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.0543425942373836